In [1]:
import pandas as pd
import numpy as np
from khaiii import KhaiiiApi
import gensim
import pyLDAvis.gensim
import numpy as np
import logging
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
import time
from pprint import pprint

In [2]:
data = pd.read_csv("./modi_data/full_data.csv") ############
#data = data.drop(['Unnamed: 0'], axis=1)
data.head()

/Users/uni613/Library/Python/3.8/lib/python/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,title,date,num,full_data
0,28920,자폐아이 치료한다고 야외에서 찬물로 씻기는 **×이 학대인정 해라,2021-02-12,"2,348",자폐아이 치료한다고 야외에서 찬물로 씻기는 **×이 학대인정 해라 안녕하세요 지금 ...
1,28919,교원임용 2차시험의 공정성과 투명성,2021-02-12,421,교원임용 2차시험의 공정성과 투명성 저는 2021학년도 중등임용시험에 응시한 수험생...
2,28918,교사 임용시험. 공정한 시험 실시 및 평가를 요구합니다.,2021-02-12,"4,092",교사 임용시험. 공정한 시험 실시 및 평가를 요구합니다. 안녕하세요. 2021 중등...
3,28917,어린이집 및 유치원 실시간 cctv 공개 의무 법안 요구,2021-02-12,963,어린이집 및 유치원 실시간 cctv 공개 의무 법안 요구 울산 인천 등등 수없이 많...
4,28916,2021년 교육. 2020년과 달라야합니다.,2021-02-09,446,2021년 교육. 2020년과 달라야합니다. 2020년 모두가 힘든 시기였습니다. ...


dfWordList = pd.read_excel("./khaiii_word_cor.xlsx")
dfWordList2 = pd.read_excel("./khaiii_word_cor_etc.xlsx")
#print(dfWordList2)

dfWordDel = dfWordList[dfWordList["수정"] == "삭제"]
dfWordMod = dfWordList[dfWordList["수정"] != "삭제"]
dfWordDiv = dfWordList2
#print(dfWordMod)

seriesDelete = dfWordDel["기존"]
stopword = []
for word in seriesDelete.values:
    stopword.append(word)
#print(stopword)

seriesModify = dfWordMod["기존"]
modiword = []
for word in seriesModify.values:
    modiword.append(word)
#print(len(modiword))

seriesModify2 = dfWordMod["수정"]
modiword2 = []
for word in seriesModify2.values:
    modiword2.append(word)
#print(len(modiword2))

seriesDivide = dfWordDiv["기존"]
divword = []
for word in seriesDivide.values:
    divword.append(word)
#print(divword)
#print(len(divword))

seriesDivide2 = dfWordDiv["수정"]
divword2 = []
for words in seriesDivide2.values:
    divword2.append(words.split(', '))
#print(divword2)
#print(len(divword2))

In [3]:
api = KhaiiiApi()
def khaiiiTokenizer(raw, pos=['NNG', 'NNP', 'NNB', 'NP', 'NR', 'SL']):#, stopword=stopword,): # 일반명사 고유명사 의존명사 대명사 수사 외국어
    list = []
    skip = 0

    for word in api.analyze(raw): #raw data
        #print(word)
        
        for i, morph in enumerate(word.morphs):
            #print(morph.lex)

            if len(morph.lex) > 1 and morph.tag in pos: #and morph.lex not in stopword: 
                if morph.tag == 'SL':
                    morph.lex = morph.lex.lower()
                # if morph.lex in divword:
                #     morph.lex = divword2[divword.index(morph.lex)]
                #     list.extend(morph.lex)
                # elif morph.lex in modiword:
                #     morph.lex = modiword2[modiword.index(morph.lex)]
                #     list.append(morph.lex)
                # else: 
                list.append(morph.lex)
                
    return list

/Users/uni613/Library/Python/3.8/lib/python/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
tokenized = data['full_data'].apply(lambda row: khaiiiTokenizer(row))
#print(tokenized)
tokenized.to_excel("./final_data/0327token_full.xlsx") ##############
print("========= tokenization completed =========")

/Users/uni613/Library/Python/3.8/lib/python/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
========= tokenization completed =========


In [5]:
#lda
id2word = gensim.corpora.Dictionary(tokenized)

/Users/uni613/Library/Python/3.8/lib/python/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
wordlist = []
for i in range(len(id2word)):
    #print(id2word[i])
    wordlist.append(id2word[i])
#print(wordlist)
seriesWordlist = pd.Series(wordlist)
seriesWordlist.to_excel("./final_data/0327wordlist_full.xlsx") #################

corpus=[id2word.doc2bow(text) for text in tokenized]
#print("id2word for each document : ", corpus)
print("# words in total : ", len(id2word))
print("# documents : ", len(corpus))

/Users/uni613/Library/Python/3.8/lib/python/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


IllegalCharacterError: 